In [64]:
import load_env
import src.graphql.models.recipe_models_autogen as rma
from sqlalchemy import select
from src.graphql.db.session import get_session
from src.graphql.scalars.recipe_scalar import NEI_Item, NEI_Fluid

In [63]:
# Test joins
recipe_id = 'r~--eEYPHsN5GqWygrEHan4A=='

stmt = select(rma.Fluid, rma.RecipeFluidGroup.fluid_inputs_key, rma.FluidGroupFluidStacks.fluid_stacks_amount) \
    .join(rma.FluidGroupFluidStacks, rma.FluidGroupFluidStacks.fluid_stacks_fluid_id == rma.Fluid.id) \
    .join(rma.RecipeFluidGroup, rma.RecipeFluidGroup.fluid_inputs_id == rma.FluidGroupFluidStacks.fluid_group_id) \
    .filter(rma.RecipeFluidGroup.recipe_id == recipe_id)
print(stmt)

SELECT fluid.id, fluid.density, fluid.fluid_id, fluid.gaseous, fluid.image_file_path, fluid.internal_name, fluid.localized_name, fluid.luminosity, fluid.mod_id, fluid.nbt, fluid.temperature, fluid.unlocalized_name, fluid.viscosity, recipe_fluid_group.fluid_inputs_key, fluid_group_fluid_stacks.fluid_stacks_amount 
FROM fluid JOIN fluid_group_fluid_stacks ON fluid_group_fluid_stacks.fluid_stacks_fluid_id = fluid.id JOIN recipe_fluid_group ON recipe_fluid_group.fluid_inputs_id = fluid_group_fluid_stacks.fluid_group_id 
WHERE recipe_fluid_group.recipe_id = :recipe_id_1


In [65]:
def _prepORMDict(item, rename={}, exclude=[], include={}):
    d = dict(item.__dict__)
    d.pop('_sa_instance_state')

    for k, v in rename.items():
        d[v] = d.pop(k)
    for k in exclude:
        d.pop(k)
    d.update(include)
    
    return d


async with get_session() as session:
    rows = await session.execute(stmt)
    item_inputs = []
    for r in rows:
        itemORM, position, liters = r
        item_inputs.append(
            NEI_Fluid(**_prepORMDict(
                itemORM,
                include={'liters': liters, 'position': position, 'input': True},
            ))
        )

    # print(result)
    # for r in result:
    #     print(r.__dict__)

    print(item_inputs)

[]
